# Lab | Unsupervised learning intro
### Instructions
It's the moment to perform clustering on the songs you collected. Remember that the ultimate goal of this little project is to improve the recommendations of artists. Clustering the songs will allow the recommendation system to limit the scope of the recommendations to only songs that belong to the same cluster - songs with similar audio features.

The experiments you did with the Spotify API and the Billboard web scraping will allow you to create a pipeline such that when the user enters a song, you:

1. Check whether or not the song is in the Billboard Hot 200.

2. Collect the audio features from the Spotify API.

After that, you want to send the Spotify audio features of the submitted song to the clustering model, which should return a cluster number.

We want to have as many songs as possible to create the clustering model, so we will add the songs you collected to a bigger dataset available on Kaggle containing 160 thousand songs.



# Import Libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time 

from dotenv import load_dotenv
import os

import warnings
warnings.filterwarnings("ignore")

<div class="alert alert-block alert-info">
<h1>Part I : Preparing Data</h1>
</div>

# Load Data
- This dataset is the audio feature from the previous lab

In [2]:
df = pd.concat(map(pd.read_csv, ['audio_features_249.csv', 
                                 'audio_features_viralhits_200.csv',
                                 'audio_features_viralhits_75.csv']))
df.reset_index(drop=True)
print("Total songs: ", len(df))
df.head(3)

Total songs:  524


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist_id,artist_name
0,0.550,0.750,0,-3.289,1,0.1460,0.0561,0.000000,0.1290,0.381,...,audio_features,5JVA0t7r2Y7m9NaHmgaeiC,spotify:track:5JVA0t7r2Y7m9NaHmgaeiC,https://api.spotify.com/v1/tracks/5JVA0t7r2Y7m...,https://api.spotify.com/v1/audio-analysis/5JVA...,147410,4,Remedy,2NpPlwwDVYR5dIj0F31EcC,Leony
1,0.714,0.425,1,-8.064,1,0.0809,0.4030,0.000105,0.1150,0.316,...,audio_features,0RSZ8EmUPEN3ySfCgytPke,spotify:track:0RSZ8EmUPEN3ySfCgytPke,https://api.spotify.com/v1/tracks/0RSZ8EmUPEN3...,https://api.spotify.com/v1/audio-analysis/0RSZ...,165477,4,Auf & Ab,5ZY4M2aGiTaZQEP6HfqeJc,Montez
2,0.761,0.525,11,-6.900,1,0.0944,0.4400,0.000007,0.0921,0.531,...,audio_features,6CDzDgIUqeDY5g8ujExx2f,spotify:track:6CDzDgIUqeDY5g8ujExx2f,https://api.spotify.com/v1/tracks/6CDzDgIUqeDY...,https://api.spotify.com/v1/audio-analysis/6CDz...,238805,4,Heat Waves,4yvcSjfu4PC0CYQyLy4wSq,Glass Animals


# Get tracks from playlists

- We have retrieved 249 audio features from the previous lab
- Since we want to cluster songs for recommendation, I prefer to get a few more. So I listed 3 playlist from spotify
- In this section, we'll retrieve 150 more songs from 3 playlists

In [3]:
# Get dotenv environment
load_dotenv() # We'll get environment variables in the .env file to use when called 

client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

# Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [4]:
def get_audio_features(playlist_id):
    
    """
    This function returns audio features from the given a playlist_id on spotify
    """
    # Get 50 tracks from playlist
    playlist = sp.user_playlist_tracks("spotify", playlist_id)
    
    # Extract track_id, title, artist_id, artist_name
    track_ids = [playlist['items'][i]['track']['id'] for i in range(playlist['total'])]
    title = [playlist['items'][i]['track']['name'] for i in range(playlist['total'])]
    
    artist_id = [playlist['items'][i]['track']['artists'][0]['id'] for i in range(playlist['total'])]
    artist_name = [playlist['items'][i]['track']['artists'][0]['name'] for i in range(playlist['total'])]
    
    # extract the audio features
    audio_features = sp.audio_features(track_ids)
    
    # store audio features in a dataframe
    df = pd.DataFrame(audio_features)
    df['title'] = title
    df['artist_id'] = artist_id
    df['artist_name'] = artist_name
    
    return df

In [5]:
df_results = []

def audio_features_multiple_playlist(playlist_id_list):
    """
    In case you have multiple playlist,
    this function takes an argument of a list of playlist_id &
    return a list of audio features
    """
    for playlist_id in playlist_id_list:
        playlist_id = get_audio_features(playlist_id)
        # df_result = pd.concat([df_result, playlist_id])
        df_results.append(playlist_id)
        time.sleep(10)


In [6]:
# Set up multiple playlists
viral50 = "37i9dQZEVXbLiRSasKsNU9"
top50_daily = "37i9dQZEVXbMDoHDwVN2tF"
top50_weekly = "37i9dQZEVXbNG2KDcFcKOF"
playlist_list = [viral50, top50_daily, top50_weekly]

In [7]:
# Calling function to get audio features from multiple playlists
audio_features_multiple_playlist(playlist_list)

# Turn the result in to one dataframe
df_result = pd.DataFrame()

for m in df_results:
    df_result = df_result.append(m)
    # df = df.append(df_res)

In [8]:
# Concat the dataset we have got from the previous lab together
df = pd.concat([df, df_result]).reset_index(drop=True)

In [9]:
# Check df
print(len(df))
df.head()

674


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist_id,artist_name
0,0.550,0.750,0,-3.289,1,0.1460,0.05610,0.000000,0.1290,0.381,...,audio_features,5JVA0t7r2Y7m9NaHmgaeiC,spotify:track:5JVA0t7r2Y7m9NaHmgaeiC,https://api.spotify.com/v1/tracks/5JVA0t7r2Y7m...,https://api.spotify.com/v1/audio-analysis/5JVA...,147410,4,Remedy,2NpPlwwDVYR5dIj0F31EcC,Leony
1,0.714,0.425,1,-8.064,1,0.0809,0.40300,0.000105,0.1150,0.316,...,audio_features,0RSZ8EmUPEN3ySfCgytPke,spotify:track:0RSZ8EmUPEN3ySfCgytPke,https://api.spotify.com/v1/tracks/0RSZ8EmUPEN3...,https://api.spotify.com/v1/audio-analysis/0RSZ...,165477,4,Auf & Ab,5ZY4M2aGiTaZQEP6HfqeJc,Montez
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,...,audio_features,6CDzDgIUqeDY5g8ujExx2f,spotify:track:6CDzDgIUqeDY5g8ujExx2f,https://api.spotify.com/v1/tracks/6CDzDgIUqeDY...,https://api.spotify.com/v1/audio-analysis/6CDz...,238805,4,Heat Waves,4yvcSjfu4PC0CYQyLy4wSq,Glass Animals
3,0.774,0.792,10,-4.021,1,0.0523,0.05100,0.000000,0.1550,0.507,...,audio_features,3eJH2nAjvNXdmPfBkALiPZ,spotify:track:3eJH2nAjvNXdmPfBkALiPZ,https://api.spotify.com/v1/tracks/3eJH2nAjvNXd...,https://api.spotify.com/v1/audio-analysis/3eJH...,139672,4,Acapulco,07YZf4WDAMNwqr4jfgOZ8y,Jason Derulo
4,0.762,0.766,7,-3.955,1,0.0343,0.00776,0.000070,0.1280,0.442,...,audio_features,5fwSHlTEWpluwOM0Sxnh5k,spotify:track:5fwSHlTEWpluwOM0Sxnh5k,https://api.spotify.com/v1/tracks/5fwSHlTEWplu...,https://api.spotify.com/v1/audio-analysis/5fwS...,287120,4,Pepas,329e4yvIujISKGKz1BZZbO,Farruko


In [10]:
# Check number of duplicated
df.duplicated().sum()

117

In [11]:
# Do a quick check on the song that duplicate
# df[df.duplicated()]
df[df['id'].str.contains('6tAKikIvnoWfUeZrfkopLL')] # pick one random duplicated song to see how it duplicates

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist_id,artist_name
220,0.79,0.979,11,-4.637,0,0.117,0.0177,0.000192,0.237,0.898,...,audio_features,6tAKikIvnoWfUeZrfkopLL,spotify:track:6tAKikIvnoWfUeZrfkopLL,https://api.spotify.com/v1/tracks/6tAKikIvnoWf...,https://api.spotify.com/v1/audio-analysis/6tAK...,146087,4,Friesenjung,6CP5wWvO8oIxedESJNCN4H,Ski Aggu
551,0.79,0.979,11,-4.637,0,0.117,0.0177,0.000192,0.237,0.898,...,audio_features,6tAKikIvnoWfUeZrfkopLL,spotify:track:6tAKikIvnoWfUeZrfkopLL,https://api.spotify.com/v1/tracks/6tAKikIvnoWf...,https://api.spotify.com/v1/audio-analysis/6tAK...,146087,4,Friesenjung,6CP5wWvO8oIxedESJNCN4H,Ski Aggu


In [12]:
# Drop duplicated rows
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
# We have total 355 songs with audio_features for clustering
len(df)

557

<div class="alert alert-block alert-info">
<h1>Part II : Clustering the songs: Unsupervised Learning</h1>
</div>

In [14]:
# Import Libraries
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min

# Select the features needed for clustering

In [15]:
# Get list of column names & copy them to create a feature list
df.columns.tolist()

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'id',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'title',
 'artist_id',
 'artist_name']

In [16]:
# Now we have 11 features from the df as an X datafram
X = df[['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']]

In [17]:
# Check data
X.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.550,0.750,0,-3.289,1,0.1460,0.05610,0.000000,0.1290,0.381,172.124
1,0.714,0.425,1,-8.064,1,0.0809,0.40300,0.000105,0.1150,0.316,99.034
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,80.870
3,0.774,0.792,10,-4.021,1,0.0523,0.05100,0.000000,0.1550,0.507,122.062
4,0.762,0.766,7,-3.955,1,0.0343,0.00776,0.000070,0.1280,0.442,130.001


In [18]:
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-0.969962,0.376694,-1.418667,1.384523,0.882923,0.477282,-0.690947,-0.231343,-0.462197,-0.539552,1.732506
1,0.283420,-1.582143,-1.150712,-0.801689,0.882923,-0.241654,0.854975,-0.230468,-0.568862,-0.826397,-1.109648
2,0.642621,-0.979424,1.528831,-0.268757,0.882923,-0.092566,1.019862,-0.231287,-0.743336,0.122397,-1.815967
3,0.741975,0.629836,1.260877,1.049380,0.882923,-0.557500,-0.713675,-0.231343,-0.264104,0.016485,-0.214189
4,0.650264,0.473129,0.457014,1.079598,0.882923,-0.756284,-0.906369,-0.230761,-0.469816,-0.270360,0.094525


In [19]:
# Clustering songs using Kmeans
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X_scaled)
clusters = kmeans.predict(X_scaled)

In [20]:
#create new dataframe with title, artist and cluster assigned
df_clustered = pd.DataFrame(X_scaled)
df_clustered['song_name'] = df['title']
df_clustered['artist'] = df['artist_name']
df_clustered['cluster'] = clusters
df_clustered.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_name,artist,cluster
0,-0.969962,0.376694,-1.418667,1.384523,0.882923,0.477282,-0.690947,-0.231343,-0.462197,-0.539552,1.732506,Remedy,Leony,1
1,0.283420,-1.582143,-1.150712,-0.801689,0.882923,-0.241654,0.854975,-0.230468,-0.568862,-0.826397,-1.109648,Auf & Ab,Montez,2
2,0.642621,-0.979424,1.528831,-0.268757,0.882923,-0.092566,1.019862,-0.231287,-0.743336,0.122397,-1.815967,Heat Waves,Glass Animals,2
3,0.741975,0.629836,1.260877,1.049380,0.882923,-0.557500,-0.713675,-0.231343,-0.264104,0.016485,-0.214189,Acapulco,Jason Derulo,1
4,0.650264,0.473129,0.457014,1.079598,0.882923,-0.756284,-0.906369,-0.230761,-0.469816,-0.270360,0.094525,Pepas,Farruko,1


In [21]:
len(df_clustered)

557

In [22]:
# Check out a song
df_clustered[df_clustered['song_name'].str.contains('Mari')]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_name,artist,cluster
444,0.764903,-0.521358,-0.882758,0.174895,0.882923,0.256411,-0.760021,-0.214583,-1.179902,0.779933,-1.153317,Maria Maria (feat. The Product G&B),Santana,1
484,1.032393,-0.786554,0.189059,-0.120873,-1.132602,-0.447064,-0.134344,-0.230359,-0.195534,0.325394,0.596928,Maria Hiwaga,Sassa Gurl,1


In [23]:
# Check out cluster == 2
df_clustered[df_clustered.cluster==2]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_name,artist,cluster
1,0.283420,-1.582143,-1.150712,-0.801689,0.882923,-0.241654,0.854975,-0.230468,-0.568862,-0.826397,-1.109648,Auf & Ab,Montez,2
2,0.642621,-0.979424,1.528831,-0.268757,0.882923,-0.092566,1.019862,-0.231287,-0.743336,0.122397,-1.815967,Heat Waves,Glass Animals,2
5,0.688477,-1.244621,-1.418667,-1.010467,-1.132602,-0.248280,0.101845,-0.231343,0.604457,1.137385,0.875154,Savage Love (Laxed - Siren Beat),Jawsh 685,2
9,0.604408,-1.558035,-0.078895,-0.808557,-1.132602,-0.809293,-0.895941,-0.226432,-0.279342,-1.814907,-1.266202,APOLOGIZE,Baby Keem,2
11,-0.045211,-0.310406,0.189059,-0.825955,-1.132602,0.002409,1.354091,-0.227916,-0.134582,-1.064698,-0.256807,Where Are You Now,Lost Frequencies,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,-2.399124,-1.787068,-0.882758,-0.968803,0.882923,-0.824754,2.646445,-0.231332,-0.568862,-0.230643,-1.174081,Jiwa Yang Bersedih,Ghea Indrawari,2
524,-2.850036,-2.088427,-0.882758,-2.559816,0.882923,-0.744136,-0.702979,1.928316,-0.401245,-1.874923,0.474049,Spider-Man 2099 (Miguel O'Hara),Daniel Pemberton,2
540,-0.282130,-0.473140,-1.150712,-0.641443,-1.132602,2.222166,1.251594,-0.231323,-0.286961,-0.486596,0.167941,Flooded The Face,Lil Uzi Vert,2
542,-1.627224,-1.630361,-1.418667,-1.388189,-1.132602,-0.852363,-0.334881,-0.047897,-0.702956,-0.107079,-2.334780,I Wanna Be Yours,Arctic Monkeys,2
